In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from feature_selection import filter_method as ft

## Отбор признаков

**Отбор признаков** - это процесс выбора подмножества признаков для обучения при построении модели машинного обучения.

#### Зачем важен отбор признаков?


Не всегда правда, что чем больше данных, тем лучше будет результат. Включение нерелевантных признаков (те, которые просто не помогают в прогнозе) и избыточных признаков (нерелевантных в присутствии других) только сделает процесс обучения перегруженным и увеличит вероятность переобучения.
С помощью выбора признаков мы можем достичь следующих целей:
- упрощение моделей для их более легкого интерпретации;
- сокращение времени обучения и снижение вычислительных затрат;
- уменьшение затрат на сбор данных;
- избегание "проклятия размерности";
- улучшение обобщения путем снижения переобучения.

Следует помнить, что разные подмножества признаков обеспечивают оптимальную производительность для разных алгоритмов. Поэтому это не отдельный процесс, а часть обучения модели машинного обучения. Если мы выбираем признаки для линейной модели, лучше использовать процедуры выбора, связанные с линейными моделями, например, размер весов или Lasso. А если мы выбираем признаки для обучения деревьев, лучше использовать признаки отобранные с помощью алгоритмов с деревьями.

### Загрузка данных

In [3]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
data = pd.DataFrame(np.c_[data['data'], data['target']],
                  columns= np.append(data['feature_names'], ['target']))

In [4]:
data.head(5)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0.0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0.0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0.0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0.0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0.0


In [5]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['target'], axis=1), 
                                                    data.target, test_size=0.2,
                                                    random_state=0)
X_train.shape, X_test.shape

((455, 30), (114, 30))

### Метод дисперсий

Удаление признаков, у которых практически все значения одинаковые

In [6]:
quasi_constant_feature = ft.constant_feature_detect(data=X_train,threshold=0.9)

0  переменных считаются почти постоянными


In [7]:
# создадим пример признака, где значения являются практически одинаковыми
X_train['dummy'] = np.floor(X_train['worst smoothness']*10)
# > 92% значений признака обладают значением 1
X_train['dummy'].value_counts() / np.float(len(X_train))

/var/folders/hg/wyjbzhnn71d60tgsx5l8yt1c0000gn/T/ipykernel_66080/1459092674.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train['dummy'].value_counts() / np.float(len(X_train))


1.0    0.923077
0.0    0.068132
2.0    0.008791
Name: dummy, dtype: float64

In [8]:
quasi_constant_feature = ft.constant_feature_detect(data=X_train,threshold=0.9)
quasi_constant_feature

1  переменных считаются почти постоянными


['dummy']

In [9]:
# удалим это значение
X_train.drop(labels=quasi_constant_feature,axis=1,inplace=True)
print(X_train.shape)

(455, 30)


## Метод корреляций

Удаление признаков, которые сильно скоррелированны друг с другом

In [10]:
corr = ft.corr_feature_detect(data=X_train,threshold=0.9)
for i in corr:
    print(i,'\n')

          feature1         feature2      corr
0   mean perimeter      mean radius  0.998185
7   mean perimeter        mean area  0.986692
14  mean perimeter  worst perimeter  0.970507
19  mean perimeter     worst radius  0.969520
32  mean perimeter       worst area  0.941920 

           feature1      feature2      corr
12  perimeter error  radius error  0.978323
30  perimeter error    area error  0.944995 

               feature1              feature2      corr
36  mean concave points        mean concavity  0.914627
41  mean concave points  worst concave points  0.906312 

         feature1      feature2      corr
38  worst texture  mean texture  0.908182 



* Коррелированные признаки не обязательно влияют на производительность модели (при использовании алгоритмов на деревьях, например), но высокая размерность влияет, а слишком большое количество функций ухудшает интерпретируемость модели. Поэтому всегда лучше уменьшить количество коррелирующих функций.